# Jupyter Notebook 4: Out of Distribution Prediction

Kang Dataset

In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sys
import util_loss as ul
#import the package to use
import beta_vae
import dhsic_vae
from dentate_features import *
from all_obs_linear_classifier_package import *
import os,glob

Using TensorFlow backend.


In [3]:
data = sc.read("./data/kang_seurat_normalized.h5ad")
data_train_full = sc.read("./data/kang_seurat_normalized_train.h5ad")
data_validate_full = sc.read("./data/kang_seurat_normalized_validate.h5ad")
cells = list(set(data_train_full.obs["cell_type"]))
print(cells)

['CD4 Naive T', 'Mk', 'B Activated', 'pDC', 'CD4 Memory T', 'T activated', 'B', 'CD14 Mono', 'DC', 'NK', 'CD16 Mono', 'CD8 T']


In [ ]:
for cell in cells:

    data_train_full_temp = data_train_full[-(data_train_full.obs["cell_type"]==cell)]
    data_validate_full_temp = data_validate_full[-(data_validate_full.obs["cell_type"]==cell)]
    print(data_train_full_temp.obs["cell_type"].value_counts())

    data_train_full_temp = ul.shuffle_adata(data_train_full_temp)
    data_validate_full_temp = ul.shuffle_adata(data_validate_full_temp)
    
    #Declaring parameters
    z = 5
    al = 100
    c = 500

    mod_path1 = "./models_kang_leave_one/latent"+str(z)+"_alpha"+str(al)+"_c"+str(c)+"_"+cell
    scg_model = beta_vae.VAEArithKeras(x_dimension= data_train_full.shape[1],z_dimension=z, 
                                          model_to_use =mod_path1,alpha=al,c_max=c)                  
    scg_model.train(data_train_full_temp,validation_data=data_validate_full_temp,
                    n_epochs=2,shuffle=False)


In [ ]:
'''
Manipulating Latent Space dimensions
simulate_multiple_cell() recovers the dropped cell type by manipulating dimensions from every cell type.
'''

#Reloading pre-trained data
from simulate_cell import *

path = "latent5_alpha20_c30_CD4 Naive T"
cell_to_drop = 'CD4 Naive T'
scg_model = beta_vae.VAEArithKeras(x_dimension= data.shape[1],z_dimension=5,model_to_use=path,
                                       alpha=20,c_max=30)
scg_model.restore_model()

data_temp = data[-(data.obs["cell_type"]==cell_to_drop)]
simulate_multiple_cell(path=path,data=data,model=scg_model,z_dim=5,feature="cell_type")


In [8]:
'''
To limit the range of simulation
It can range between mean value of the Source cell.
Latent Representation plots can be used to limit the manipulation to a more local range to generate clearer plots.
'''

path = "latent5_alpha20_c30_CD4 Naive T"
cell_to_drop = 'CD4 Naive T'
scg_model = beta_vae.VAEArithKeras(x_dimension= data.shape[1],z_dimension=5,model_to_use=path,
                                       alpha=20,c_max=30)
scg_model.restore_model()

data_temp = data[-(data.obs["cell_type"]==cell_to_drop)]
feature = "cell_type"
cell = "B"
z_dim = 5

variable_names = data.var_names
data_latent = scg_model.to_latent(data.X)
latent_df = pd.DataFrame(data_latent)
latent_df[feature] = list(data.obs[feature])
latent_df["condition"] = list(data.obs["condition"])
try:
    os.makedirs(path+"/gene_heatmaps/")
except OSError:
    pass
x_dim = data.shape[1]
data_ast = latent_df[latent_df[feature]==cell]
data_ast = data_ast[data_ast["condition"]=="CTRL"]
cell_one = data_ast.iloc[[0],[0,1,2,3,4]]

for dim in range(z_dim):
    '''
    a and b are the ranges for simulation. Change it according to local requirements. 
    a = data_ast[dim].mean()
    '''
    a = min(data_latent[:,dim]) 
    b = max(data_latent[:,dim])
    increment_range = np.arange(a,b,0.01)
    result_array = np.empty((0, x_dim))
    for inc in increment_range:
            cell_latent = cell_one
            #print(cell_latent)
            #print(cell_latent.shape)
            cell_latent.iloc[:,dim] = inc
            cell_recon = scg_model.reconstruct(cell_latent)
            result_array = np.append(result_array,cell_recon,axis=0)

    result_adata = sc.AnnData(result_array, obs={"inc_vals":increment_range},var={"var_names":variable_names})
    result_adata.write(path+"/gene_heatmaps/"+str(cell)+"_"+str(dim)+".h5ad")



<tf.Variable 'Variable_4:0' shape=() dtype=float32>
Model: "VAE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 14053)             0         
_________________________________________________________________
encoder (Model)              (None, 5)                 11896810  
_________________________________________________________________
decoder (Model)              (None, 14053)             11906853  
Total params: 23,803,663
Trainable params: 23,797,263
Non-trainable params: 6,400
_________________________________________________________________
<tf.Variable 'Variable_4:0' shape=() dtype=float32>


In [ ]:
'''
Creating Regression plots, UMAPS and PCA plots
'''

from seurat_umaps_reg_plots import *

sc.tl.rank_genes_groups(data, groupby="cell_type", method='t-test')
x_dim = data.shape[1]

path = "latent5_alpha20_c30_CD4 Naive T"
cell_to_drop = 'CD4 Naive T'
cells = list(set(data.obs["cell_type"]))

scg_model = beta_vae.VAEArithKeras(x_dimension= data.shape[1],z_dimension=5,model_to_use=path,
                                       alpha=20,c_max=30)
scg_model.restore_model()

df_list = generate_simulated_reg_plots(path=path+"/gene_heatmaps/",
                            actual_data=data,clust_typ = cell_to_drop,cells=cells)
df = pd.DataFrame(df_list,columns=["name","r_sq_all","r_sq_100"])
df.to_csv(path+"/reg_mean.csv",index=False)

In [ ]:
'''
Directly using PCA 
'''

sim_data = sc.read("latent5_alpha20_c30_CD4 Naive T/gene_heatmaps/B_4.h5ad")
print(sim_data)
generate_simulated_pca(path = "latent5_alpha20_c30_CD4 Naive T/gene_heatmaps/",actual_data = data,clust_typ = "CD4 Naive T",source_cell="B",sim_data=sim_data,first_cell="B")

Dentate Gyrus

In [ ]:
data = sc.read("./data/dentate_gyrus_normalized.h5ad")
#data_train_full = sc.read("./data/dentate_gyrus_normalized_train.h5ad")
#data_validate_full = sc.read("./data/dentate_gyrus_normalized_validate.h5ad")
cells = list(set(data_train_full.obs["clusters"]))
print(cells)

In [ ]:
for cell in cells:

    data_train_full_temp = data_train_full[-(data_train_full.obs["clusters"]==cell)]
    data_validate_full_temp = data_validate_full[-(data_validate_full.obs["clusters"]==cell)]
    print(data_train_full_temp.obs["clusters"].value_counts())

    data_train_full_temp = ul.shuffle_adata(data_train_full_temp)
    data_validate_full_temp = ul.shuffle_adata(data_validate_full_temp)
    
    #Declaring parameters
    z = 5
    al = 100
    c = 30

    mod_path1 = "./models_dentate_leave_one/latent"+str(z)+"_alpha"+str(al)+"_c"+str(c)+"_"+cell
    scg_model = beta_vae.VAEArithKeras(x_dimension= data_train_full.shape[1],z_dimension=z, 
                                          model_to_use =mod_path1,alpha=al,c_max=c)                  
    scg_model.train(data_train_full_temp,validation_data=data_validate_full_temp,
                    n_epochs=2,shuffle=False)


In [ ]:
'''
Manipulating Latent Space dimensions
'''

#Reloading pre-trained data
from simulate_cell import *

path = "latent5_alpha100_c30_Granule immature"
cell_to_drop = 'Granule immature'
scg_model = beta_vae.VAEArithKeras(x_dimension= data.shape[1],z_dimension=5,model_to_use=path,
                                       alpha=100,c_max=30)
scg_model.restore_model()

data_temp = data[-(data.obs["clusters"]==cell_to_drop)]
simulate_multiple_cell(path=path,data=data,model=scg_model,z_dim=5,feature="clusters")


In [ ]:
'''
To limit the range of simulation
It can range between mean value of the Source cell.
Latent Representation plots can be used to limit the manipulation to a more local range to generate clearer plots.
'''

path = "latent5_alpha100_c30_Granule immature"
cell_to_drop = 'Granule immature'
scg_model = beta_vae.VAEArithKeras(x_dimension= data.shape[1],z_dimension=5,model_to_use=path,
                                       alpha=100,c_max=500)
scg_model.restore_model()

data_temp = data[-(data.obs["clusters"]==cell_to_drop)]
feature = "clusters"
cell = "Neuroblast"
z_dim = 5

variable_names = data.var_names
data_latent = scg_model.to_latent(data.X)
latent_df = pd.DataFrame(data_latent)
latent_df[feature] = list(data.obs[feature])
try:
    os.makedirs(path+"/gene_heatmaps/")
except OSError:
    pass
x_dim = data.shape[1]
data_ast = latent_df[latent_df[feature]==cell]
cell_one = data_ast.iloc[[0],[0,1,2,3,4]]

for dim in range(z_dim):
    '''
    a and b are the ranges for simulation. Change it according to local requirements. 
    a = data_ast[dim].mean()
    '''
    a = min(data_latent[:,dim]) 
    b = max(data_latent[:,dim])
    increment_range = np.arange(a,b,0.01)
    result_array = np.empty((0, x_dim))
    for inc in increment_range:
            cell_latent = cell_one
            #print(cell_latent)
            #print(cell_latent.shape)
            cell_latent.iloc[:,dim] = inc
            cell_recon = scg_model.reconstruct(cell_latent)
            result_array = np.append(result_array,cell_recon,axis=0)

    result_adata = sc.AnnData(result_array, obs={"inc_vals":increment_range},var={"var_names":variable_names})
    result_adata.write(path+"/gene_heatmaps/"+str(cell)+"_"+str(dim)+".h5ad")


In [ ]:
sc.tl.rank_genes_groups(data, groupby="clusters", method='t-test')
x_dim = data.shape[1]

In [ ]:
'''
Creating Regression plots, UMAPS and PCA plots
'''

from dentate_umaps_reg_plots import *

path = "latent5_alpha100_c30_Granule immature"
cell_to_drop = 'Granule immature'
cells = list(set(data.obs["clusters"]))

scg_model = beta_vae.VAEArithKeras(x_dimension= data.shape[1],z_dimension=5,model_to_use=path,
                                       alpha=50,c_max=30)
scg_model.restore_model()

df_list = generate_simulated_reg_plots(path=path+"/gene_heatmaps/",
                            actual_data=data,clust_typ = cell_to_drop,cells=cells)
df = pd.DataFrame(df_list,columns=["name","r_sq_all","r_sq_100"])
df.to_csv(path+"/gene_heatmaps/reg_mean.csv",index=False)